In [1]:
# import required libraries

import pandas as pd
import numpy as np
import seaborn as sns

In [ ]:
# This line is used to create html version of the notebook.

In [ ]:
import os

os.system('jupyter nbconvert --to html HW2.ipynb')